In [1]:
import numpy as np
import scipy
from scipy.sparse import coo_matrix

def count_nonzero_coo(matrix):
    return len(matrix.data)

n = 1000
num_matrices = 10000
total_nonzero_elements = 0

for i in range(num_matrices):

    random_state = 3893 + i
    rng = np.random.RandomState(random_state)

    zero_prob = rng.uniform(0.995, 0.999)
    nnz = int((1 - zero_prob) * n ** 2)
    rows = [rng.randint(0, n) for _ in range(nnz)]
    cols = [rng.randint(0, n) for _ in range(nnz)]

    uniques = set(zip(rows, cols))  # Ensure we do not have repeated edges
    rows, cols = zip(*uniques)

    # Generate values for non-zero elements
    vals = np.array([rng.normal(0, 1) for _ in cols])
    A = coo_matrix((vals, (rows, cols)), shape=(n, n))

    # Set diagonal elements to a random number different from 0
    diag_indices = np.arange(n)
    diag_vals = rng.normal(0, 2, size=n)  # Random values for the diagonal elements
    diag_vals[diag_vals == 0] = rng.uniform(1, 2)  # Replace 0s with a random number different than 0

    A = A + coo_matrix((diag_vals, (diag_indices, diag_indices)), shape=(n, n))
    A = coo_matrix(A)
    nnz = count_nonzero_coo(A)
    total_nonzero_elements += nnz

average_nonzero_elements = total_nonzero_elements / num_matrices
print("Average number of nonzero elements:", average_nonzero_elements)



Average number of nonzero elements: 3969.9775


In [54]:
import numpy as np
import scipy
from scipy.sparse import coo_matrix, csr_matrix, rand
import pyamg

def count_nonzero_coo(matrix):
    return len(matrix.data)

n = 2500
num_matrices = 1
total_nonzero_elements = 0

n = int(np.sqrt(n))

for i in range(num_matrices):


    random_state = 3343 + i
    rng = np.random.RandomState(random_state)

    A = pyamg.gallery.poisson((n, n))
    A = csr_matrix(A)

    # Find the non-zero elements in the matrix
    non_zero_indices = A.nonzero()

    num_non_zero_elements = len(non_zero_indices[0])

    # Determine the number of elements to perturb
    perturb_percentage = rng.uniform(0.99, 0.99)
    num_elements_to_perturb = int(perturb_percentage * num_non_zero_elements)

    # Select a random subset of the non-zero elements
    selected_indices = rng.choice(num_non_zero_elements, num_elements_to_perturb, replace=False)

    # Generate random values for the selected non-zero elements
    perturbation_values = rng.normal(0, 1, size=num_elements_to_perturb)

    # Perturb the selected non-zero elements with different random values
    A.data[selected_indices] += perturbation_values

    # # Generate a random sparse matrix with a density between 0.5% and 2.5%
    density = rng.uniform(0.025, 0.025)
    random_sparse_matrix = scipy.sparse.random(A.shape[0], A.shape[1], density=density, format='csr', random_state=random_state)

    # Add the random sparse matrix to A
    A += random_sparse_matrix

    A = coo_matrix(A)



    nnz = count_nonzero_coo(A)
    total_nonzero_elements += nnz
    # print(total_nonzero_elements)

average_nonzero_elements = total_nonzero_elements / num_matrices
print("Average number of nonzero elements:", average_nonzero_elements)

Average number of nonzero elements: 168228.0
